In [ ]:
from diffusion_libs import *
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [ ]:
fill_vocabulary()

In [ ]:
def format_output(sample):
  return ' '.join(str(element) for element in sample)

In [ ]:
checkpoint_path = "checkpoints\\diffusion_model\\cp-0032\\model"

# sampling
min_signal_rate = 0.02
max_signal_rate = 0.95

# architecture
embedding_dims = 32
embedding_max_frequency = 1000.0
embedding_min_frequency = 1.0

# optimization
batch_size = 64
ema = 0.999
learning_rate = 1e-3
weight_decay = 1e-4

# dictionary related
DICTIONARY_SIZE = 246
TOKENS_CAPACITY = 2048

widths = [32, 64, 96, 128]
block_depth = 2

parsed_dir = "./data/parsed/"
max_size = 64
dataset, filenames = load_dataset(parsed_dir, max_size)
dataset = scale_dataset_down(dataset, DICTIONARY_SIZE)
    
network = get_network(TOKENS_CAPACITY, widths, block_depth, embedding_min_frequency, embedding_max_frequency, embedding_dims)
print("Network created")

model = DiffusionModel(TOKENS_CAPACITY, DICTIONARY_SIZE, network, batch_size, max_signal_rate, min_signal_rate, ema)
print("Model created")

model.compile(
    optimizer = keras.optimizers.experimental.AdamW(learning_rate=learning_rate, weight_decay=weight_decay),
    loss = keras.losses.mean_absolute_error
)
model.normalizer.adapt(dataset)

model.load_weights(checkpoint_path)


In [ ]:
raw, denormalized = model.generate(5,100)

In [ ]:
print(max(raw[0]))
print(min(raw[0]))

In [ ]:
print(max(denormalized[0]))
print(min(denormalized[0]))

In [ ]:
denormalized

In [ ]:
print(format_output(convert_back_to_code(denormalized[0])))